In [1]:
using Pkg
Pkg.add("Distributions")
Pkg.add("FFTW")
Pkg.add("Plots")

using Distributions
using FFTW
using Plots

   Updating registry at `C:\Users\saman\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `C:\Users\saman\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\saman\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\saman\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\saman\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\saman\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\saman\.julia\environments\v1.4\Manifest.toml`
 [no changes]


In [2]:
num_photons = 100
num_pixels  = 256

function heat(A::Array{Int64,2}, title::String, label="Photons")
    Plots.heatmap(A, title=title, ylabel=label)
end

heat (generic function with 2 methods)

In [3]:
# Ground truth: a circular phase-only object
numPixels = 512
gtRadius  = 50.0 # ground truth radius, pixels
gtPhase   = 0.75 # radians
gtCenter  = numPixels / 2 # assumes numPixels is even

W = [convert(Float64, y) for x=0:numPixels-1, y=0:numPixels-1]
H = [convert(Float64, x) for x=0:numPixels-1, y=0:numPixels-1]

# masking
gt = ones(ComplexF64, numPixels, numPixels)
for x=1:numPixels, y=1:numPixels
    dist = sqrt((W[x, y] - gtCenter)^2 + (H[x, y] - gtCenter)^2)
    if dist <= gtRadius
        gt[x, y] = exp(1.0im * gtPhase)
    end
end

# Physical dimensions and sampling
pixelSize = 0.1 # microns

x = LinRange(-pixelSize * numPixels / 2, pixelSize * numPixels / 2, numPixels)
dx = x[2] - x[1]              # Sampling period, microns
fS = 1 / dx                      # Spatial sampling frequency, inverse microns
df = fS / numPixels              # Spacing between discrete frequency coordinates, inverse microns
fx = collect(-fS/2 + df:df:fS/2) # Spatial frequency, inverse microns

# Fourier transform of ground truth
GT = ifftshift(fft(fftshift(gt))) * dx^2

# TODO: Wird nur in animate verwendet, wird das gebraucht?
# Angular spectrum propagator and spatial frequencies
function H_FN(fx::Array{Float64,2}, fy::Array{Float64,2}, z::Float64, wavelength::Float64=0.5)
    term = 1 .- (wavelength^2 .* fx.^2) - (wavelength^2 .* fy.^2)
    result = [if (m_ij < 0) 0 + 0im else exp(1im * 2 * pi * z / wavelength * sqrt(m_ij)) end for  m_ij in term]
    return result
end

# All rows have the fx vector
# a b
# a b
FX = [x for y=1:length(fx),x in fx]

# All columns have the fx vector
# a a 
# b b
FY = [x for x in fx, y=1:length(fx)]

# Field at a distance of z=0
gt_prime = fftshift(ifft(ifftshift(GT))) / dx^2

# Normalizing constant: makes the maximum photon count at z = 0 equal to 100 photons
maxVal, _ = findmax([abs(x)^2 for x in gt_prime])
norm = maxVal / 100

0.010000000000000014

In [4]:
function add_camera_noise(input_irrad_photons, qe=0.69, sensitivity=5.88, dark_noise=2.29, bitdepth=12, baseline=100)
    # Add shot noise
    width, height = size(input_irrad_photons)
    photons = [rand(Poisson(m_ij)) for m_ij in input_irrad_photons]
    
    # Convert to electrons
    electrons = qe * photons
    
    # Add dark noise
    electrons_out = [rand(Normal(dark_noise)) for x=1:width,y=1:height] + electrons
    
    # Convert to ADU and add baseline
    max_adu = 2^bitdepth - 1
    adu =  round.(Int, electrons_out .* sensitivity)
    adu .+= baseline
    clamp!(adu, 0, max_adu)
    
    return adu
end

add_camera_noise (generic function with 6 methods)

In [ ]:
numPoints = 100
z = range(0, length=100, stop=numPoints)
vmin, vmax = 0, 1500

function render_frame(frame::Integer)
    gt_prime = fftshift(ifft(ifftshift(GT .* H_FN(FX, FY, z[frame], 0.525)))) / dx^2
    
    # Divide by norm to convert to a photons
    hologram = [abs(m_ij)^2 / norm for m_ij in gt_prime]
    
    adu = add_camera_noise(hologram)

    print("z=$(round(frame/numPoints*100))%\r")
    return adu
end

anim = @animate for i=1:numPoints
    heat(render_frame(i), "dark_noise=default", "ADU")
end
gif(anim, "animation.gif", fps = 10)

In [ ]:
numPoints = 100
z = range(0, length=100, stop=numPoints)
vmin, vmax = 0, 1500

function render_frame(frame::Integer)
    gt_prime = fftshift(ifft(ifftshift(GT .* H_FN(FX, FY, z[frame], 0.525)))) / dx^2
    
    # Divide by norm to convert to a photons
    hologram = [abs(m_ij)^2 / norm for m_ij in gt_prime]
    
    adu = add_camera_noise(hologram, 0.69, 5.88, 20.0, 12, 100)

    print("z=$(round(frame/numPoints*100))%\r")
    return adu
end

anim = @animate for i=1:numPoints
    heat(render_frame(i), "dark_noise=20.0", "ADU")
end
gif(anim, "animation2.gif", fps = 10)